In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:13 http://ppa

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-10 01:08:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.58MB/s    in 0.2s    

2022-01-10 01:08:52 (5.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"]).dropDuplicates((["review_id"]))
vine_df.show()
vine_df.dtypes

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R101CJ0JBI9J43|          5|            6|          6|   N|                Y|
|R101P7GUI5O1WG|          5|            0|          0|   N|                Y|
|R102LL91IX39PK|          5|            0|          0|   N|                Y|
|R102YVSWWEAP5K|          5|            0|          0|   N|                Y|
|R1042WRU1WDUHY|          5|            7|          8|   N|                Y|
|R104G7VC1XPVOO|          5|            1|          1|   N|                Y|
|R10557FTU9GHY8|          5|            0|          0|   N|                Y|
|R105B7Y5ALJ45D|          5|            1|          1|   N|                Y|
|R105C41QCCFLIN|          5|            0|          0|   N|                Y|
|R105MO44HU33PJ|          5|            0|          0|   N|     

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [ ]:
#Filter to retrieve all rows where total_votes >=20
filtered_vine_df = vine_df.filter("total_votes >=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R11YMYHSPNGA9|          5|          112|        117|   N|                Y|
|R12E1DPQO0PSC5|          5|           55|         57|   N|                N|
|R12Q28Q3H61LJ4|          5|           47|         47|   N|                Y|
|R12ZZBLV20KHZN|          5|           21|         24|   N|                Y|
|R14RP0JIFZOU0F|          1|            9|         26|   N|                Y|
|R15ERI6B6F5POS|          5|           27|         29|   N|                Y|
|R162U5PSTTFDRY|          5|           30|         31|   N|                N|
|R17HY9US9BLH1T|          1|           75|         81|   N|                Y|
|R18E8DQJPSW932|          4|           25|         26|   N|                N|
|R18YQ6WBZM7CTR|          1|           25|         25|   N|     

In [ ]:
#Create DataFrame of rows where helpful_votes / total_votes =>50%
helpful_df = filtered_vine_df.filter("(helpful_votes / total_votes)>=.5")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R11YMYHSPNGA9|          5|          112|        117|   N|                Y|
|R12E1DPQO0PSC5|          5|           55|         57|   N|                N|
|R12Q28Q3H61LJ4|          5|           47|         47|   N|                Y|
|R12ZZBLV20KHZN|          5|           21|         24|   N|                Y|
|R15ERI6B6F5POS|          5|           27|         29|   N|                Y|
|R162U5PSTTFDRY|          5|           30|         31|   N|                N|
|R17HY9US9BLH1T|          1|           75|         81|   N|                Y|
|R18E8DQJPSW932|          4|           25|         26|   N|                N|
|R18YQ6WBZM7CTR|          1|           25|         25|   N|                Y|
|R19V1JU0BUV5K4|          5|           46|         48|   N|     

In [ ]:
#Create DataFrame of rows where a review was part of a Vine program (vine == "Y")
YVineProg_df = helpful_df.filter("vine == 'Y'")
YVineProg_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2XLAQL0E2UPWQ|          4|           33|         34|   Y|                N|
|R3F6KXGIFXQ3B6|          4|           40|         43|   Y|                N|
| R761OO343LFQW|          3|           82|         93|   Y|                N|
|R1XZXTP2UJR37K|          4|          127|        128|   Y|                N|
|R1241UCQGRHCNH|          5|           72|         76|   Y|                N|
|R1CN8C3E6GNUFO|          1|           29|         33|   Y|                N|
|R2D1YV9RG1BD7K|          4|           18|         22|   Y|                N|
|R380KQVSEH3PQT|          5|           26|         28|   Y|                N|
| RAZ7HBRW7G0AX|          5|          313|        333|   Y|                N|
|R21MHBZMJWQL1K|          4|           34|         36|   Y|     

In [ ]:
#Create DataFrame of rows where a review was not part of a Vine program (vine == "N")
NVineProg_df = helpful_df.filter("vine == 'N'")
NVineProg_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R11YMYHSPNGA9|          5|          112|        117|   N|                Y|
|R12E1DPQO0PSC5|          5|           55|         57|   N|                N|
|R12Q28Q3H61LJ4|          5|           47|         47|   N|                Y|
|R12ZZBLV20KHZN|          5|           21|         24|   N|                Y|
|R15ERI6B6F5POS|          5|           27|         29|   N|                Y|
|R162U5PSTTFDRY|          5|           30|         31|   N|                N|
|R17HY9US9BLH1T|          1|           75|         81|   N|                Y|
|R18E8DQJPSW932|          4|           25|         26|   N|                N|
|R18YQ6WBZM7CTR|          1|           25|         25|   N|                Y|
|R19V1JU0BUV5K4|          5|           46|         48|   N|     

In [ ]:
#Determine total number of reviews, number of 5-star reviews, and percentage of 5-star reviews - Paid
YTotal_reviews = YVineProg_df.count()
YTotal_reviews

170

In [ ]:
#Determine total number of reviews, number of 5-star reviews, and percentage of 5-star reviews - Paid
YFiveStar = YVineProg_df.filter("star_rating = 5").count()
YFiveStar

65

In [ ]:
#Determine total number of reviews, number of 5-star reviews, and percentage of 5-star reviews - Paid
YPercent = YFiveStar / YTotal_reviews
YPercent

0.38235294117647056

In [ ]:
#Determine total number of reviews, number of 5-star reviews, and percentage of 5-star reviews - UnPaid
NTotal_reviews = NVineProg_df.count()
NTotal_reviews

37840

In [ ]:
#Determine total number of reviews, number of 5-star reviews, and percentage of 5-star reviews - UnPaid
NFiveStar = NVineProg_df.filter("star_rating = 5").count()
NFiveStar

20612

In [ ]:
#Determine total number of reviews, number of 5-star reviews, and percentage of 5-star reviews - UnPaid
NPercent = NFiveStar / NTotal_reviews
NPercent

0.5447145877378435